In [212]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
from collections import Counter

# Métodos de Validación
from sklearn.model_selection import train_test_split

# Modelos de Clasificación
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Métricas para Clasificación
from sklearn.metrics import jaccard_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# OverSampling y UnderSampling
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

In [213]:
df = pd.read_pickle(r'C:\Users\José\Desktop\proyecto_aviones\data\pickle\vuelos_limpio.pkl')

In [214]:
# Creamos la columna target y ya la binarizamos
df["llega_tarde"] = np.where(df['retraso_llegada'] > 15, 'si', 'no')
df['llega_tarde'] = df['llega_tarde'].map({'si': 1, 'no': 0})

In [215]:
# Eliminamos las columnas que no aportan información extra y retraso en la llegada para que no prediga el 100% de las veces y mes que solo tiene 1 valor
columnas_eliminar = [
    "latitude_destino",
    "latitude_origen",
    "longitude_destino",
    "longitude_origen",
    "codigo_aeropuerto_destino",
    "codigo_aeropuerto_origen",
    "direccion_destino",
    "direccion_origen",
    "retraso_llegada",
    "hora_llegada_real",
    "mes"
]
df.drop(columns=columnas_eliminar, inplace=True)


In [216]:
diccionario_costo = {
    'alto_costo': ["Delta Air Lines", "American Airlines", "United Airlines", "Alaska Airlines", "Hawaiian Airlines"],
    'medio_costo': ["Southwest Airlines", "JetBlue Airways"],
    'bajo_costo': ['Allegiant Air', 'Frontier Airlines', 'Spirit Airlines', 'Envoy Air', 'SkyWest Airlines', 'PSA Airlines', 
                'Endeavor Air', 'Mesa Airlines', 'Republic Airways', 'Horizon Air']}

def asignar_costo(aerolinea):
    for costo, aerolineas in diccionario_costo.items():
        if aerolinea in aerolineas:
            return costo

df['aerolinea'] = df['aerolinea'].apply(asignar_costo)

In [217]:
pesos_categorias = {'alto_costo': 2, 'medio_costo': 1, 'bajo_costo': 0} 

df['aerolinea'] = df['aerolinea'].map(pesos_categorias)

In [218]:
# Cambiamos a minutos las columnas horarias

df["hora_salida_real"] = df["hora_salida_real"].apply(lambda x: x.hour * 60 + x.minute)
df["hora_salida_programada"] = df["hora_salida_programada"].apply(lambda x: x.hour * 60 + x.minute)

df['hora_despegue'] = df['hora_despegue'].replace('24:00', '00:00')
df['hora_despegue'] = pd.to_datetime(df['hora_despegue'], format='%H:%M').dt.hour * 60 + pd.to_datetime(df['hora_despegue'], format='%H:%M').dt.minute

In [219]:
# Como da error el datetime de fecha, extraemos el día del mes, que es el único que nos falta, porque mes como todos tienen el mismo no hace falta
# y sobbreescribimos la columna de fecha, 

df['fecha'] = df['fecha'].dt.day

In [220]:
# Miramos las columnas object que tenemos que cambiar a numéricas para el modelo
columnas_categoricas = df.select_dtypes(include=["object"]).columns.tolist()

In [221]:
def target_encoding(df, columna_categorica, target):

    media_columna_con_target = df.groupby(columna_categorica)[target].mean()
    df[columna_categorica] = df[columna_categorica].map(media_columna_con_target)
    return

# target_encoding(df, 'aerolinea', 'llega_tarde') esta ya no xk la he puesto un label encoding
target_encoding(df, 'numero_cola', 'llega_tarde')
target_encoding(df, 'ciudad_origen', 'llega_tarde')
target_encoding(df, 'estado_origen', 'llega_tarde')
target_encoding(df, 'aeropuerto_origen', 'llega_tarde')
target_encoding(df, 'ciudad_destino', 'llega_tarde')
target_encoding(df, 'estado_destino', 'llega_tarde')
target_encoding(df, 'aeropuerto_destino', 'llega_tarde')

In [222]:
# aplico logaritmo

df["distancia_millas"] = np.log(df["distancia_millas"] + 1) 

In [223]:
# Definimos la target

TARGET = "llega_tarde"
X = df.drop(columns = TARGET)
y  = df[[TARGET]]

In [224]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

X_train: (1353980, 26), y_train: (1353980, 1)
X_test: (338496, 26), y_test: (338496, 1)


In [225]:
# Analizamos la distribución de clases en el conjunto de datos. 

contador_clases= Counter([arr[0] for arr in list(y_train.values)])
print(contador_clases)

total = sum(contador_clases.values())
for clase, count in contador_clases.items():
    porcentaje = (count / total) * 100
    print(f'Porcentaje de "{clase}": {porcentaje:.2f}%')

Counter({0: 1248772, 1: 105208})
Porcentaje de "0": 92.23%
Porcentaje de "1": 7.77%


In [226]:
# Definir las estrategias de balanceo
undersampling = RandomUnderSampler(sampling_strategy=0.1)
oversampling = SMOTE(sampling_strategy=0.6)

# Definir la pipeline
balance_pipeline = Pipeline([
    ('undersampling', undersampling),
    ('oversampling', oversampling)
])

# Aplicar la pipeline al conjunto de entrenamiento
X_balanceado, y_balanceado = balance_pipeline.fit_resample(X_train, y_train)

In [227]:
# Comprobamos que se han balanceado más 
contador_clases= Counter([arr[0] for arr in list(y_balanceado.values)])
print(contador_clases)

total = sum(contador_clases.values())
for clase, count in contador_clases.items():
    porcentaje = (count / total) * 100
    print(f'Porcentaje de "{clase}": {porcentaje:.2f}%')

Counter({0: 1052080, 1: 631248})
Porcentaje de "0": 62.50%
Porcentaje de "1": 37.50%


In [228]:
def test_models(X, y):
    # Definir modelos
    models = {
        "Random Forest"       : RandomForestClassifier(),
        "Naive Bayes"         : GaussianNB(),
        "KNN"                 : KNeighborsClassifier(),
        "Gradient Boosting"   : GradientBoostingClassifier(),
        'Nearest Centroid'    : NearestCentroid(),
        'Decision Tree'       : DecisionTreeClassifier(),
        'AdaBoost Classifier' : AdaBoostClassifier()
    }

    # DataFrame para almacenar los resultados de las métricas
    results = list()

    # Entrenar y evaluar cada modelo
    for name, model in models.items():
        model.fit(X, y)
        y_pred = model.predict(X_test)

        # Calcular métricas
        jaccard_index = jaccard_score(y_test, y_pred, average="macro")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average="macro")
        recall = recall_score(y_test, y_pred, average="macro")
        f1_score_value = f1_score(y_test, y_pred, average="macro")
        roc_auc = roc_auc_score(y_test, y_pred)
        conf_matrix = confusion_matrix(y_test, y_pred, labels=[0, 1])
        tn, fp, fn, tp = conf_matrix.ravel()
        specificity = tn / (tn + fp)

        # Agregar resultados a la lista
        results.append({
            'Model': name,
            'Jaccard Index': jaccard_index,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1-score': f1_score_value,
            'ROC AUC': roc_auc,
            'Specificity': specificity
        })

    # Convertir la lista de resultados a un DataFrame
    df_results = pd.DataFrame(results)

    return df_results

df_resultados = test_models(X_balanceado, y_balanceado)

df_resultados

C:\Users\José\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\José\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\José\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change 

,Model,Jaccard Index,Accuracy,Precision,Recall,F1-score,ROC AUC,Specificity
0,Random Forest,0.889988,0.981861,0.922004,0.958588,0.939363,0.958588,0.986143
1,Naive Bayes,0.492686,0.798630,0.592960,0.724478,0.605496,0.724478,0.812274
2,KNN,0.743817,0.943072,0.789327,0.919280,0.838564,0.919280,0.947449
3,Gradient Boosting,0.833966,0.968475,0.860888,0.962611,0.903790,0.962611,0.969554
4,Nearest Centroid,0.299281,0.526110,0.525651,0.589481,0.422933,0.589481,0.514449
5,Decision Tree,0.890983,0.982083,0.923466,0.958133,0.939963,0.958133,0.986489
6,AdaBoost Classifier,0.790969,0.957429,0.827657,0.942174,0.873940,0.942174,0.960236


In [229]:
df_resultados

,Model,Jaccard Index,Accuracy,Precision,Recall,F1-score,ROC AUC,Specificity
0,Random Forest,0.889988,0.981861,0.922004,0.958588,0.939363,0.958588,0.986143
1,Naive Bayes,0.492686,0.798630,0.592960,0.724478,0.605496,0.724478,0.812274
2,KNN,0.743817,0.943072,0.789327,0.919280,0.838564,0.919280,0.947449
3,Gradient Boosting,0.833966,0.968475,0.860888,0.962611,0.903790,0.962611,0.969554
4,Nearest Centroid,0.299281,0.526110,0.525651,0.589481,0.422933,0.589481,0.514449
5,Decision Tree,0.890983,0.982083,0.923466,0.958133,0.939963,0.958133,0.986489
6,AdaBoost Classifier,0.790969,0.957429,0.827657,0.942174,0.873940,0.942174,0.960236


In [ ]:
fig = px.bar(df_resultados, x="Model", y=["Jaccard Index", "Accuracy", "Precision", "Recall", "F1-score", "ROC AUC", "Specificity"],
             title="Métricas de Evaluación de los Modelos de Aprendizaje Automático",
             labels={"value": "Valor de la Métrica", "variable": "Métrica"},
             barmode="group")

fig.update_layout(title_x=0.5, xaxis = {'categoryorder' : 'total descending'})

fig.show()

C:\Users\José\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



# Observaciones.

- Vemos que el modelo que mejor métrica `ROC AUC` nos da es **Grandient Boosting**, sin embargo en términos generales de precisión y especificidad (`Precision` & `Sprecifity`) además de `Accuracy` es **Decision Tree**, 

    por lo que usaremos ese modelo para clasificar nuestros datos.